# Day 1: Introduction to AmberTools & Small Molecule Parameterization

This notebook introduces **AmberTools**, its sub-programs, and how they are used for
building and parameterizing small organic molecules for molecular dynamics workflows.

## What is AmberTools?
AmberTools is an open‑source suite used for:
- Preparing molecules for MD simulations
- Generating force‑field parameters (GAFF, GAFF2)
- File manipulation & conversions
- Geometry optimization and charge assignment

## Key AmberTools programs covered today

**1. antechamber**
- Generates atom types (GAFF/GAFF2)
- Computes partial charges (AM1-BCC, RESP)
- Produces parameter files (`.mol2`, `.prepi`, `.frcmod`)

**2. parmchk2**
- Checks for missing parameters
- Generates a `.frcmod` file containing supplemental parameters

**3. tleap**
- Assembles the full system (molecule, solvent, ions)
- Writes AMBER `prmtop` and `inpcrd`

We will also visualize the molecule using **nglview**.

In [ ]:
# Install nglview (if needed)
import sys
import os
!{sys.executable} -m pip install nglview > /dev/null
import nglview as nv
from rdkit import Chem
from rdkit.Chem import AllChem
import parmed as pmd

PWD=os.getcwd()
print(PWD)

## Step 1: Load a small molecule (ethanol example)
We will start from a SMILES string and convert it into a 3D structure using OpenBabel.

In [ ]:
%mkdir -p {PWD}/small_molecule
%cd {PWD}/small_molecule

# Example: Generate 3D MOL2 from SMILES using OpenBabel
# Ensure OpenBabel is installed in your conda environment
# !conda install -c conda-forge openbabel -y

# Convert SMILES to 3D Tripos MOL2
smiles = "CCO"  # ethanol
mol_name = "ethanol"
!obabel -:{smiles} -O {mol_name}.mol2 --gen3d

## Visualize the molecule

In [ ]:
# Load from MOL2 file
view = nv.show_file(f"{mol_name}.mol2")
view

## Step 2: Run `antechamber` to create GAFF atom types and AM1-BCC charges
```bash
antechamber options help:
Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -nc    net molecular charge (int)
                   -rn    residue name

```

In [ ]:
!antechamber -i {mol_name}.mol2 -fi mol2 -o {mol_name}_gaff.mol2 -fo mol2 -at gaff2 -c bcc -s 2 -pf y
!tail  sqm.out

In [ ]:
!ls -l

## Step 3: Generate supplemental parameters using `parmchk2`
```bash
Usage: parmchk2 -i    input file name
                -o    frcmod file name
                -f    input file format (prepi, prepc, ac, mol2, frcmod, leaplog) 
```

In [ ]:
!parmchk2 -i {mol_name}_gaff.mol2 -f mol2 -o {mol_name}.frcmod
!cat ethanol.frcmod

## Step 5: Build topology & coordinates with `tleap`

Create a file **tleap_{mol_name}.in**:

Run:
```bash
tleap -f tleap_{mol_name}.in
```

In [ ]:
with open("tleap.in","w+") as f:
    f.writelines(f'''
source leaprc.gaff2
source leaprc.water.tip3p

# Load molecule (from antechamber-generated files)
ETH = loadmol2 {mol_name}_gaff.mol2
loadamberparams {mol_name}.frcmod

# Build topology and coordinate files
saveamberparm ETH {mol_name}.prmtop {mol_name}.inpcrd

# Optional: PDB for visualization
savepdb ETH {mol_name}.pdb''')

In [ ]:
!tleap -f tleap.in

## Step 6: Convert to GROMACS formats

In [ ]:

# Load AMBER topology & coordinates
amber = pmd.load_file(f"{mol_name}.prmtop", f"{mol_name}.inpcrd")

# Save GROMACS structure (.gro)
amber.save(f"{mol_name}.gro", overwrite=True)

# Save GROMACS topology (.top)
amber.save(f"topol.top", format="gromacs", overwrite=True)

print("Conversion completed: molecule.gro + topol.top")

In [ ]:
#!cat topol.top

# Monomer parameterisation

## Step 1: Prepare the monomer from SMILES

In [ ]:
%mkdir -p {PWD}/monomer
%cd {PWD}/monomer
#PEO: polyethylene oxide polymer
#Use shor polymer with repeat group, head and tail group
#repeat group: COCCOCCOCCOCCOC (5 monomers) 
#head and tail group: C

# Convert SMILES to 3D Tripos MOL2
smiles = "CCOCCOCCOCCOCCOCC"  
mol_name = "PEO"
!obabel -:{smiles} -O {mol_name}.mol2 --gen3d
# Load from MOL2 file
view = nv.show_file(f"{mol_name}.mol2")
view.add_label(radius=2,color='black',label_type='atomindex')
view


## Step 2: Run antechamber to create GAFF atom types and AM1-BCC charges

In [ ]:
!antechamber -i {mol_name}.mol2 -fi mol2 -o {mol_name}_gaff.ac -fo ac -at gaff2 -an y -c bcc -nc 0 -rn PEO  -s 2 -pf y
!tail sqm.out